In [46]:
from app_store_scraper import AppStore
from google_play_scraper import app, Sort, reviews_all, reviews

import pandas as pd
import numpy as np
import datetime

scrape from App store

In [8]:
app_inst = AppStore(country="ng", app_name="instagram")
app_inst.review(after=datetime.datetime(2022, 12,31)) #scrape all after Dec 31 2022

2023-03-29 14:57:18,154 [INFO] Base - Searching for app id
2023-03-29 14:57:20,702 [INFO] Base - Initialised: AppStore('ng', 'instagram', 389801252)
2023-03-29 14:57:20,703 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/ng/app/instagram/id389801252
2023-03-29 14:57:25,893 [INFO] Base - [id:389801252] Fetched 9 reviews (9 fetched in total)
2023-03-29 14:57:31,610 [INFO] Base - [id:389801252] Fetched 21 reviews (21 fetched in total)
2023-03-29 14:57:37,431 [INFO] Base - [id:389801252] Fetched 32 reviews (32 fetched in total)
2023-03-29 14:57:43,617 [INFO] Base - [id:389801252] Fetched 46 reviews (46 fetched in total)
2023-03-29 14:57:50,207 [INFO] Base - [id:389801252] Fetched 54 reviews (54 fetched in total)
2023-03-29 14:57:56,040 [INFO] Base - [id:389801252] Fetched 62 reviews (62 fetched in total)
2023-03-29 14:58:02,010 [INFO] Base - [id:389801252] Fetched 68 reviews (68 fetched in total)
2023-03-29 14:58:07,591 [INFO] Base - [id:389801252] Fetched 75 reviews (75 

In [9]:
#save to df
temp_df = pd.DataFrame(app_inst.reviews); temp_df.head()
temp_df['type'] = 'app_store'
temp_df['country'] = 'ng'

issues scraping from US

In [14]:
us_inst = AppStore(country="us", app_name="instagram")
us_inst.review(after=datetime.datetime(2022, 12,31), sleep=10) #scrape all after Dec 31 2022

2023-03-29 17:36:55,222 [INFO] Base - Searching for app id
2023-03-29 17:36:57,464 [INFO] Base - Initialised: AppStore('us', 'instagram', 389801252)
2023-03-29 17:36:57,465 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/instagram/id389801252
2023-03-29 17:37:08,364 [INFO] Base - [id:389801252] Fetched 2 reviews (2 fetched in total)
2023-03-29 17:37:30,063 [INFO] Base - [id:389801252] Fetched 4 reviews (4 fetched in total)
2023-03-29 17:37:52,236 [INFO] Base - [id:389801252] Fetched 5 reviews (5 fetched in total)
2023-03-29 17:38:24,683 [ERROR] Base - Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/389801252/reviews?l=en-GB&offset=120&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 404 error responses'))
2023-03-29 17:38:24,687 [INFO] Base - [id:389801252] Fetched 6 reviews (6 fetched in total)


In [53]:
#save to df
us_df = pd.DataFrame(us_inst.reviews); us_df.head()
us_df['type'] = 'app_store'
us_df['country'] = 'us'

In [54]:
appdf = pd.concat([temp_df, us_df])

scrape from Google play

In [65]:
ggplay_reviews = reviews(
    'com.instagram.android',
    lang = 'en',
    country='us', # Not getting other countries
    sort=Sort.NEWEST,
    count = 12000,
)

In [71]:
ggplay_reviews2 = reviews(
    'com.instagram.android',
    lang = 'en',
    country='us', # Not getting other countries
    sort=Sort.NEWEST,
    count = 12000,
    continuation_token=ggplay_reviews[1]
)

In [75]:
ggplay_reviews3 = reviews(
    'com.instagram.android',
    lang = 'en',
    country='us', # Not getting other countries
    sort=Sort.NEWEST,
    count = 12000,
    continuation_token=ggplay_reviews2[1]
)

In [79]:
ggdf = pd.DataFrame(np.array(ggplay_reviews[0]),columns=['review'])
ggdf = pd.DataFrame(ggdf.review.tolist())
ggdf['type'] = 'play_store'

In [82]:
ggdf3 = pd.DataFrame(np.array(ggplay_reviews3[0]),columns=['review'])
ggdf3 = pd.DataFrame(ggdf3.review.tolist())
ggdf3['type'] = 'play_store'

In [83]:
ggdf2 = pd.DataFrame(np.array(ggplay_reviews2[0]),columns=['review'])
ggdf2 = pd.DataFrame(ggdf2.review.tolist())
ggdf2['type'] = 'play_store'

In [84]:
ggdf = ggdf.rename(columns={'at':'date', 'content':'review', 'score':'rating'})
ggdf2 = ggdf.rename(columns={'at':'date', 'content':'review', 'score':'rating'})
ggdf3 = ggdf.rename(columns={'at':'date', 'content':'review', 'score':'rating'})

In [86]:
gg_play=pd.concat([ggdf, ggdf2, ggdf3])

In [87]:
#filterout date outside range
gg_play = gg_play[gg_play.date > datetime.datetime(2022,12,31)]

concatenate and save to disk

In [88]:
df = pd.concat([gg_play, appdf]).drop(['reviewId','userImage','isEdited'], axis=1)

In [89]:
df.shape

(36517, 11)

In [91]:
df.to_csv("insta_reviews_df.csv", index=False)

# NB
>> it seems gg_play_scraper broke and hence cannot filter by country so it scrapes globally i.e from all countries. due to this some non english reviews might be included in the reviews too